In [44]:
!pip install transformers -q
!pip install pandas -q
!pip install scikit-learn -q
!pip install openpyxl -q
!pip install tabulate -q
!pip install PatternLite -q
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
import os
from data import *

In [13]:
# check if Bertified data exists
if not os.path.isfile('../data/Bertified/entities.npy'):
    reverb_lines = read_reverb('../data/Reverb1.1/reverb_wikipedia_tuples-1.1.txt')
    questions = pd.read_excel('../data/ReverbSQA/Final_Sheet_990824.xlsx', sheet_name=1, engine='openpyxl')
    index = get_tuple_frequency(reverb_lines, questions)
    index[index['Frequency']<10].to_excel('../data/ProcessedQuestions/normalized_questions.xlsx')
    combine_with_reverb(questions_path='../data/ProcessedQuestions/normalized_questions.xlsx', 
                    reverb_path='../data/Reverb1.1/reverb_wikipedia_tuples-1.1.txt')
    create_bertified_dataset()


In [19]:
with open('/root/capsule/results/output.txt', 'w') as f:
    f.write('Start')

In [31]:
%%capture cap --no-stderr
# table 1 output

print('\n\n', '#'*20, "Table 1".upper(), '#'*20)
print(pd.read_excel('../data/Intermediate/train.xlsx').sample(6)[['triple', 'Question']])
with open('/root/capsule/results/output.txt', 'a') as f:
    f.write(cap.stdout)



In [32]:
%%capture cap --no-stderr
# table 2 output

print('\n\n', '#'*20, "Table 2".upper(), '#'*20)
df = pd.read_csv(r'../data/Reverb1.1/reverb_wikipedia_tuples-1.1.txt', sep='\t', header=None)
reverb_columns_name = ['ExID', 'arg1', 'rel', 'arg2', 'narg1', 'nrel', 'narg2', 'csents', 'conf', 'urls']
df.columns = reverb_columns_name
df = df.dropna()
df = df.drop_duplicates()

print(f'#Triples\t\t:\t{len(df)}')
print(f'#Relations\t\t:\t{len(df["rel"].unique())}')
print(f'#Entity 1\t\t:\t{len(df["arg1"].unique())}')
print(f'#Entity 2\t\t:\t{len(df["arg2"].unique())}')
print(f'Total Unique Entities\t:\t{len(set(df["arg1"].unique().tolist()+df["arg2"].unique().tolist()))}')
vocab = df["arg1"].unique().tolist()+df["arg2"].unique().tolist()+df["rel"].unique().tolist()
vocab = list(map(lambda x:x.split(), vocab))
vocab = [item for sublist in vocab for item in sublist]
print(f'Vocabulary Size\t\t:\t{len(set(vocab))}')
with open('/root/capsule/results/output.txt', 'a') as f:
    f.write(cap.stdout)

In [33]:
%%capture cap --no-stderr
# table 3 output

print('\n\n', '#'*20, "Table 3".upper(), '#'*20)
train_df = pd.read_excel('../data/Intermediate/train.xlsx'); valid_df = pd.read_excel('../data/Intermediate/valid.xlsx'); test_df = pd.read_excel('../data/Intermediate/test.xlsx')


def get_unique_ent_rel(dataframe):
    arg1 = [eval(item)[0] for item in dataframe['triple'].to_list()]
    arg2 = [eval(item)[2] for item in dataframe['triple'].to_list()]
    rel = [eval(item)[1] for item in dataframe['triple'].to_list()]
    print(f'Number of Questions\t:\t{len(dataframe)}')
    print(f'Entity 1\t:\t{len(set(arg1))}')
    print(f'Entity 2\t:\t{len(set(arg2))}')
    print(f'Relations\t:\t{len(set(rel))}')
    print(f'Total Unique Entities\t:\t{len(set(arg1+arg2))}')
    tokenizer = lambda string:string.strip().lower().split()
    tokenized_questions = dataframe['Question'].astype(str).apply(tokenizer).to_list()
    flatten_tokenized_questions = [item for sublist in tokenized_questions for item in sublist]
    print(f'Unique Words\t:\t{len(set(flatten_tokenized_questions))}')

print("*** Training ***")
get_unique_ent_rel(train_df)
print("\n*** Validation ***")
get_unique_ent_rel(valid_df)
print("\n*** Test ***")
get_unique_ent_rel(test_df)
with open('/root/capsule/results/output.txt', 'a') as f:
    f.write(cap.stdout)

In [37]:
%%capture cap --no-stderr

print('\n\n', '#'*20, "Table 5 GRU".upper(), '#'*20)
# table 5 output {GRU's row}
!python3 ./BuboQA/entities/train.py  --entity_detection_mode GRU \
                                    --fix_embed --data_dir ../data/SimpleQuestionNotationEntity \
                                    --batch_size 256 \
                                    --vector_cache ../data/Cache/sq_glove300d.pt \

print('\n\n', '#'*20, "Table 5 LSTM".upper(), '#'*20)
# table 5 output {GRU's row}
!python3 ./BuboQA/entities/train.py  --entity_detection_mode LSTM \
                                    --fix_embed --data_dir ../data/SimpleQuestionNotationEntity \
                                    --batch_size 256 \
                                    --vector_cache ../data/Cache/sq_glove300d.pt \

print('\n\n', '#'*20, "Table 6 LSTM".upper(), '#'*20)
# table 6 output {LSTM's row}
!python3 ./BuboQA/relations/train.py  --relation_prediction_mode LSTM \
                                     --fix_embed --data_dir ../data/SimpleQuestionNotationRelation \
                                     --batch_size 256 \
                                     --vector_cache ../data/Cache/sq_glove300d.pt

print('\n\n', '#'*20, "Table 6 CNN".upper(), '#'*20)
# table 6 output {CNN's row}
!python3 ./BuboQA/relations/train.py  --relation_prediction_mode CNN \
                                     --fix_embed --data_dir ../data/SimpleQuestionNotationRelation \
                                     --batch_size 256 \
                                     --vector_cache ../data/Cache/sq_glove300d.pt

with open('/root/capsule/results/output.txt', 'a') as f:
    f.write(cap.stdout)

In [46]:
%%capture cap --no-stderr
# table 7 & 8 & 9 & 10 output

%cd /root/capsule/code/src
print('\n\n', '#'*20, "Table 7 & 8 Test".upper(), '#'*20)
!python3 train.py False NodeEdgeDetector rsq test
print('\n\n', '#'*20, "Table 7 & 8 Valid".upper(), '#'*20)
!python3 train.py False NodeEdgeDetector rsq valid
print('\n\n', '#'*20, "Table 9 & 10 Test".upper(), '#'*20)
!python3 train.py False NodeEdgeDetector sq test
print('\n\n', '#'*20, "Table 9 & 10 Valid".upper(), '#'*20)
!python3 train.py False NodeEdgeDetector sq valid
with open('/root/capsule/results/output.txt', 'a') as f:
    f.write(cap.stdout)

/root/capsule/code/src


 #################### TABLE 7 & 8 TEST ####################
INFO:root:

############# Fold Number 1 #############


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 140076598681024 on /root/.cache/huggingface/hub/models--bert-base-uncased/blobs/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:filelock:Lock 140076598681024 acquired on /root/.cache/huggingface/hub/models--bert-base-uncased/blobs/45a2321a7ecfdaaf60a6c1fd7f5463994cc8907d.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 570
DEBUG:filelock:Attempting to release lock 140076598681024 on /root/.cache/huggingface/hub/models--bert-base-uncased/blobs/45a

In [12]:
%%capture cap --no-stderr
# table 11 output

%cd /root/capsule/code/src
print('\n\n', '#'*20, "Table 11 BERT-LSTM-CRF".upper(), '#'*20)
!python3 train.py False BertLSTMCRF rsq test
print('\n\n', '#'*20, "Table 11 BERT-CNN".upper(), '#'*20)
!python3 train.py False BertCNN rsq test
print('\n\n', '#'*20, "Table 11 Multi-Depth ".upper(), '#'*20)
!python3 train.py False MultiDepthNodeEdgeDetector rsq test
print('\n\n', '#'*20, "Table 11 Fine_tune BERT".upper(), '#'*20)
!python3 train.py False NodeEdgeDetector rsq test
with open('/root/capsule/results/output.txt', 'a') as f:
    f.write(cap.stdout)


/content/CodeOceanBERTQA/code/src
2023-03-19 17:55:57.772869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 17:55:58.686035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-19 17:55:58.686160: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-19 17:55:58.686184: W tensorflow/compiler/tf

In [48]:
%cd /root/capsule/code/src
!mkdir ../../data/Models
!python3 train.py True NodeEdgeDetector rsq test

mkdir: cannot create directory ‘../../data/Models’: File exists
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be e

In [49]:
%cd /root/capsule/code/src
!python3 evaluation.py tfidf

/root/capsule/code/src
/usr/local/lib/python3.8/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
Traceback (most recent call last):
  File "evaluation.py", line 2, in <module>
    from graph import *
  File "/root/capsule/code/src/graph.py", line 9, in <module>
    from pattern.en import conjugate, lemma, lexeme,PRESENT,SG,PAST
ModuleNotFoundError: No module named 'pattern'


In [8]:
%%capture cap --no-stderr
# table 12 output

%cd /root/capsule/code
print('\n\n', '#'*20, "Table 12 (Test, Valid)".upper(), '#'*20)

from src.utils import get_hit
import pandas as pd
test_df = pd.read_excel('../data/Intermediate/test.xlsx')
actual = test_df['Reverb_no'].to_list()
system_results = pd.read_excel('../data/Candidates/test_results.xlsx')['sys'].apply(lambda item:eval(item))
print(get_hit(actual, system_results))
test_df = pd.read_excel('../data/Intermediate/valid.xlsx')
actual = test_df['Reverb_no'].to_list()
system_results = pd.read_excel('../data/Candidates/valid_results.xlsx')['sys'].apply(lambda item:eval(item))
print(get_hit(actual, system_results))

with open('/root/capsule/results/output.txt', 'a') as f:
    f.write(cap.stdout)

(0.732560463721767, 0.8474915050969418, 0.8648810713571857, 0.878273036178293, 0.8924645212872276)
(0.7572815533980582, 0.8669331810394061, 0.8800685322672759, 0.8897772701313535, 0.903483723586522)


In [1]:
%%capture cap --no-stderr
# table 13 output

%cd /root/capsule/code
print('\n\n', '#'*20, "Table 13 (Test, Valid)".upper(), '#'*20)

import pandas as pd
from collections import Counter
test = pd.read_excel('../data/Candidates/test_results.xlsx', engine ='openpyxl')
valid = pd.read_excel('../data/Candidates/valid_results.xlsx', engine ='openpyxl')
rr = lambda row: 1/10000 if int(row['Reverb_no']) not in [item[0] for item in eval(row['sys'])] else 1/(1+[item[0] for item in eval(row['sys'])].index(row['Reverb_no']))
test['RR'] = test.apply(rr, axis=1)
valid['RR'] = valid.apply(rr, axis=1)
print(test['RR'].mean(), valid['RR'].mean())
with open('/root/capsule/results/output.txt', 'a') as f:
    f.write(cap.stdout)

0.7810465810353977 0.801262243574989


In [61]:
from thefuzz import fuzz
import pandas as pd
refrence = pd.read_excel('../data/Intermediate/test.xlsx')
refrence.Question = refrence.Question.apply(lambda x:str(x).lower().strip())
error_dataframe = {
                    'Question':[],
                    'candidates':[],
                    'actual':[],
                    'node':[],
                    'edge':[]
                }
with open('../data/Detailed/Test_Set_With.txt', 'r') as res:
    for line in res:
        if line.find('Question')!=-1:
            temp = eval(line.split(': ')[1].strip())
            error_dataframe['Question'].append(' '.join(temp))
        elif line.find('Sorted candidates')!=-1:
            error_dataframe['candidates'].append(eval(line.split(': ')[1].strip()))
        elif line.find('Node: ')!=-1:
            line = line.replace(', Edge','').split(': ')
            error_dataframe['node'].append(line[1])
            error_dataframe['edge'].append(line[2].strip())
        elif line.find('Actual line number')!=-1:
            error_dataframe['actual'].append(eval(line.split(': ')[1].strip()))
error_dataframe['Meaningful'] = [1 for _ in error_dataframe['actual']]
# for k,v in error_dataframe.items():
#     print(k, len(v), v[:5])
#     error_dataframe[k]=v[:5350]
error_df = pd.DataFrame(error_dataframe)       

In [64]:
error_analysis = pd.merge(error_df, refrence, how='inner', on='Question')
error_analysis['candidates_index'] = error_analysis['candidates'].apply(lambda item:[i[0] for i in item])
gh = lambda row:row['candidates_index'].index(row['actual']) if (row['actual'] in row['candidates_index']) else 100
error_analysis['hit'] = error_analysis.apply(gh, axis=1)
empty_condidates = error_analysis[(error_analysis['hit']<10)&(error_analysis['hit']>-1)][['Question', 'node', 'edge', 'normalized_triple', 'Reverb_no', 'hit']]
empty_condidates['triple'] = empty_condidates.normalized_triple.apply(lambda x:list(str(item).lower() for item in eval(x)))

In [65]:
node_precision, edge_precision = [], []
print(len(empty_condidates))
for index, row in empty_condidates.iterrows():
    temp = max([fuzz.ratio(item, row['node']) for item in row['triple']])
    node_precision.append(temp) 
    temp = max([fuzz.ratio(item, row['edge']) for item in row['triple']])
    edge_precision.append(temp) 
# print(min(node_precision))
print(sum(node_precision)/len(node_precision))
print(sum(edge_precision)/len(edge_precision))

3041
98.5386386057218
77.67181848076291


In [ ]:
import faiss
assert faiss.get_num_gpus() > 0

In [10]:
98.4924990852543
78.15331137943652

98.52592341579125
77.76821416468994


98.54240106418357
77.73761223811107

98.5386386057218
77.66425517921736

98.54151624548736
77.65408598621595




99.40541529454812
92.34687156970362
